In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import websocket
import json
# import os
# import math
# import configparser
# import time

In [ ]:
WS_URL = 'wss://stream.binance.us:9443/ws/btcusdt@kline_1m'

In [ ]:
class OnlineTrading(object):
    def __init__(self):
        self.data = pd.DataFrame()

    def run(self):
        ws = websocket.WebSocketApp(
            WS_URL,
            on_open=self.__on_open,
            on_message=self.__on_message,
            on_close=self.__on_close
        )
        ws.run_forever()

    def __on_open(self, ws):
        print("Connection established")

    def __on_close(self, ws, code, msg):
        print(f"Connection closed")

    def __on_message(self, ws, message):
        data = json.loads(message)
        close = float(data['k']['c'])
        date = dt.datetime.fromtimestamp(int(int(data['E']) / 1000))
        is_candle_closed = bool(data['k']['x'])

        if is_candle_closed:
            # create & append new candle to self.data
            new_candle = pd.DataFrame({ "price": close }, index=[date])
            new_candle.index.name = 'Date'
            self.data = pd.concat([self.data, new_candle])

            # re-sample in minute candle and calc returns
            dr = self.data.resample('1m', label='right').last()
            dr['return'] = np.log(dr / dr.shift(1))

            print(dr)

bot = OnlineTrading()
bot.run()